In [13]:
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import BatchNormalization
from PIL import Image
from random import shuffle, choice
import numpy as np
import os

IMAGE_SIZE = 256
IMAGE_DIRECTORY = './train'

# Generate a label for the image based on the directory it is in
def label_img(name):
  if name == 'cat': return np.array([1, 0])
  elif name == 'notcat' : return np.array([0, 1])

# Load in and preprocess the images from the directories, return an array of them with labels
def load_data():
  print("Loading images...")
  train_data = []
  directories = next(os.walk(IMAGE_DIRECTORY))[1]

  for dirname in directories:
    print("Loading {0}".format(dirname))
    file_names = next(os.walk(os.path.join(IMAGE_DIRECTORY, dirname)))[2]

    for i in range(200):
      image_name = choice(file_names)
      image_path = os.path.join(IMAGE_DIRECTORY, dirname, image_name)
      label = label_img(dirname)
      if "DS_Store" not in image_path:
        img = Image.open(image_path)
        img = img.convert('L')
        img = img.resize((IMAGE_SIZE, IMAGE_SIZE))
        train_data.append([np.array(img), label])

  return train_data

# Create the model for binary classification of whether a cat is present
def create_model():
  model = Sequential()
  model.add(Conv2D(32, kernel_size = (3, 3), activation='relu', 
                   input_shape=(IMAGE_SIZE, IMAGE_SIZE, 1)))
  model.add(MaxPooling2D(pool_size=(2,2)))
  model.add(BatchNormalization())
  model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
  model.add(MaxPooling2D(pool_size=(2,2)))
  model.add(BatchNormalization())
  model.add(Conv2D(128, kernel_size=(3,3), activation='relu'))
  model.add(MaxPooling2D(pool_size=(2,2)))
  model.add(BatchNormalization())
  model.add(Conv2D(128, kernel_size=(3,3), activation='relu'))
  model.add(MaxPooling2D(pool_size=(2,2)))
  model.add(BatchNormalization())
  model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
  model.add(MaxPooling2D(pool_size=(2,2)))
  model.add(BatchNormalization())
  model.add(Dropout(0.2))
  model.add(Flatten())
  model.add(Dense(256, activation='relu'))
  model.add(Dropout(0.2))
  model.add(Dense(64, activation='relu'))
  model.add(Dense(2, activation = 'softmax'))

  return model

# load data and convert into shape/format expected by Keras
training_data = load_data()
training_images = np.array([i[0] for i in training_data]).reshape(-1, IMAGE_SIZE, IMAGE_SIZE, 1)
training_labels = np.array([i[1] for i in training_data])

# Train model based on the provided cat/nocat images
print('creating model')
model = create_model()
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print('training model')
model.fit(training_images, training_labels, batch_size=50, epochs=10, verbose=1)
model.save("model.h5")

Loading images...
Loading cat
Loading notcat
creating model
training model
Epoch 1/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 34s 3s/step - accuracy: 0.6177 - loss: 0.7045
Epoch 2/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 20s 3s/step - accuracy: 0.8729 - loss: 0.2906
Epoch 3/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 20s 3s/step - accuracy: 0.9918 - loss: 0.0937
Epoch 4/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 21s 3s/step - accuracy: 0.9983 - loss: 0.0328
Epoch 5/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 21s 3s/step - accuracy: 0.9955 - loss: 0.0227
Epoch 6/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 21s 3s/step - accuracy: 0.9950 - loss: 0.0211
Epoch 7/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 21s 3s/step - accuracy: 0.9988 - loss: 0.0073
Epoch 8/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 21s 3s/step - accuracy: 0.9991 - loss: 0.0068
Epoch 9/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 21s 3s/step - accuracy: 0.9978 - loss: 0.0097
Epoch 10/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 21s 3s/step - accuracy: 1.0000 - loss: 0.0082


In [25]:
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import BatchNormalization
from PIL import Image
from random import shuffle, choice
import numpy as np
import os

# Set constants
IMAGE_SIZE = 256
IMAGE_DIRECTORY = './test'

# Method for labeling image based on classification/ground truth
def label_img(name):
    if name == 'cat': return np.array([1, 0])
    elif name == 'notcat' : return np.array([0, 1])

# Load in images and label
def load_data(imdir):
  print("Loading images...")
  test_data = []
  directories = next(os.walk(imdir))[1]

  for dirname in directories:
    print("Loading {0}".format(dirname))
    file_names = next(os.walk(os.path.join(imdir, dirname)))[2]
    for i in range(len(file_names)):
      image_name = choice(file_names)
      image_path = os.path.join(imdir, dirname, image_name)
      if image_name != ".DS_Store":
        label = label_img(dirname)
        img = Image.open(image_path)
        img = img.convert('L')
        img = img.resize((IMAGE_SIZE, IMAGE_SIZE))
        test_data.append([np.array(img), label])
  
  return test_data

# Load and preprocess images
test_data = load_data(IMAGE_DIRECTORY)
test_images = np.array([i[0] for i in test_data]).reshape(-1, IMAGE_SIZE, IMAGE_SIZE, 1)
test_labels = np.array([i[1] for i in test_data])

# Load in model generated in previous step
print('Loading model...')
model = load_model("model.h5")

# Assess model accuracy using Keras methods
print('Testing model...')
loss, acc = model.evaluate(test_images, test_labels, verbose=1)

print("accuracy: {0}".format(acc * 100))
print("loss: {0}".format(loss * 100))

Loading images...
Loading cat
Loading notcat
Loading model...


Testing model...


2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 205ms/step - accuracy: 0.6246 - loss: 1.5398
accuracy: 63.999998569488525
loss: 143.56509447097778
